In [ ]:
!pip install transformers
!pip install datasets
!pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()

# Load the dataset (in Colab)
df = pd.read_excel('TASK_11.xlsx')

# Display the first few rows to understand the structure
print(df.head())



Saving TASK_11.xlsx to TASK_11.xlsx
                        id                                               text  \
0  eng_train_track_a_00001                                But not very happy.   
1  eng_train_track_a_00002  Well she's not gon na last the whole song like...   
2  eng_train_track_a_00003  She sat at her Papa's recliner sofa only to mo...   
3  eng_train_track_a_00004                    Yes, the Oklahoma city bombing.   
4  eng_train_track_a_00005                       They were dancing to Bolero.   

   Anger  Fear  Joy  Sadness  Surprise  
0      0     0    1        1         0  
1      0     0    1        0         0  
2      0     0    0        0         0  
3      1     1    0        1         1  
4      0     0    1        0         0  


In [ ]:
import pandas as pd

# Load the dataset (assuming the file is uploaded or loaded in Colab)
df = pd.read_excel('TASK_11.xlsx')

# Define a function to extract the dominant emotion
def get_emotion(row):
    if row['Anger'] == 1:
        return 'Anger'
    elif row['Fear'] == 1:
        return 'Fear'
    elif row['Joy'] == 1:
        return 'Joy'
    elif row['Sadness'] == 1:
        return 'Sadness'
    elif row['Surprise'] == 1:
        return 'Surprise'
    return 'Unknown'  # In case none are marked

# Apply the function to each row and create a new 'Emotion' column
df['Emotion'] = df.apply(get_emotion, axis=1)

# Check the first few rows to verify the new Emotion column
print(df.head())


                        id                                               text  \
0  eng_train_track_a_00001                                But not very happy.   
1  eng_train_track_a_00002  Well she's not gon na last the whole song like...   
2  eng_train_track_a_00003  She sat at her Papa's recliner sofa only to mo...   
3  eng_train_track_a_00004                    Yes, the Oklahoma city bombing.   
4  eng_train_track_a_00005                       They were dancing to Bolero.   

   Anger  Fear  Joy  Sadness  Surprise  Emotion  
0      0     0    1        1         0      Joy  
1      0     0    1        0         0      Joy  
2      0     0    0        0         0  Unknown  
3      1     1    0        1         1    Anger  
4      0     0    1        0         0      Joy  


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['Emotion'], test_size=0.2, random_state=42)

# Display a sample of the split data
print(train_texts[:5])
print(train_labels[:5])


2124    I need to ice my calf often and try to stay of...
2716                    I remember being scared shitless.
2232    I began to feel the all-too-familiar feeling t...
261                                     Ill pay you back!
2059      I did settle down and dry my eyes on my sleeve.
Name: text, dtype: object
2124       Fear
2716       Fear
2232       Fear
261         Joy
2059    Sadness
Name: Emotion, dtype: object


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the emotion labels to numerical values
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

# Verify encoding
print(label_encoder.classes_)  # To see which number corresponds to which emotion


['Anger' 'Fear' 'Joy' 'Sadness' 'Surprise' 'Unknown']


In [ ]:
print(type(train_texts))  # Should be a list
print(train_texts[:5])    # Check the first few elements to ensure they are strings
print(type(test_texts))    # Should be a list
print(test_texts[:5])      # Check the first few elements to ensure they are strings


<class 'list'>
['I need to ice my calf often and try to stay off it... except to occasionally stretch it.', 'I remember being scared shitless.', 'I began to feel the all-too-familiar feeling that I my heart was being ripped from my chest.', 'Ill pay you back!', 'I did settle down and dry my eyes on my sleeve.']
<class 'list'>
['I smoke weed alone I have a tendency to become hungry.', 'Nothing but fine grey and tan sand as far as my eyes could see.', 'After an evening there we were driving back because we had no money for a hotel.', 'It never freaked me out, because it happened enough as i was small and I got used to it.', 'Only damage done was scarring and a broken collar bone.']


In [ ]:
# Filter out any non-string entries
train_texts = [text for text in train_texts if isinstance(text, str)]
test_texts = [text for text in test_texts if isinstance(text, str)]

# Verify the cleaned lists
print(len(train_texts), len(test_texts))


2210 553


In [ ]:
from transformers import DistilBertTokenizer

# Load the pre-trained DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the train and test texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')

# Print to verify encodings
print(train_encodings)


{'input_ids': tensor([[  101,  1045,  2342,  ...,     0,     0,     0],
        [  101,  1045,  3342,  ...,     0,     0,     0],
        [  101,  1045,  2211,  ...,     0,     0,     0],
        ...,
        [  101,  1045,  7948,  ...,     0,     0,     0],
        [  101, 13055,  2351,  ...,     0,     0,     0],
        [  101,  2026,  2159,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer

# Assuming df is already loaded
# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['Emotion'], test_size=0.2, random_state=42)

# Convert texts to lists if they're pandas Series
train_texts = train_texts.tolist()
test_texts = test_texts.tolist()

# Filter out any non-string entries
train_texts = [text for text in train_texts if isinstance(text, str)]
test_texts = [text for text in test_texts if isinstance(text, str)]

# Encode the emotion labels to numerical values
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

# Load the pre-trained DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the train and test texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')

# Print to verify encodings
print(train_encodings)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'input_ids': tensor([[  101,  1045,  2342,  ...,     0,     0,     0],
        [  101,  1045,  3342,  ...,     0,     0,     0],
        [  101,  1045,  2211,  ...,     0,     0,     0],
        ...,
        [  101,  1045,  7948,  ...,     0,     0,     0],
        [  101, 13055,  2351,  ...,     0,     0,     0],
        [  101,  2026,  2159,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
import torch

class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = EmotionDataset(train_encodings, train_labels)
test_dataset = EmotionDataset(test_encodings, test_labels)


In [ ]:
from transformers import DistilBertForSequenceClassification

# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


In [ ]:
print(f"Length of train encodings: {len(train_encodings['input_ids'])}")
print(f"Length of train labels: {len(train_labels)}")


Length of train encodings: 2210
Length of train labels: 2214


In [ ]:
# Check for any missing values in the original DataFrame
missing_texts = df[df['text'].isnull()]
missing_labels = df[df[['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']].isnull().any(axis=1)]

print(f"Missing texts: {missing_texts}")
print(f"Missing labels: {missing_labels}")


Missing texts:                            id text  Anger  Fear  Joy  Sadness  Surprise  \
717   eng_train_track_a_00718  NaN      0     1    0        0         1   
1537  eng_train_track_a_01538  NaN      1     0    0        0         1   
1744  eng_train_track_a_01745  NaN      0     0    0        0         0   
2570  eng_train_track_a_02571  NaN      0     1    0        0         0   
2628  eng_train_track_a_02629  NaN      0     0    0        0         1   

       Emotion  
717       Fear  
1537     Anger  
1744   Unknown  
2570      Fear  
2628  Surprise  
Missing labels: Empty DataFrame
Columns: [id, text, Anger, Fear, Joy, Sadness, Surprise, Emotion]
Index: []


In [ ]:
# Drop rows with missing texts or labels
df = df.dropna(subset=['text', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])


In [ ]:
from sklearn.model_selection import train_test_split

# Define your features and labels
X = df['text']
y = df[['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']].values.tolist()

# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels into one-hot encoding if necessary
train_labels = [list(label) for label in train_labels]  # Convert to list of lists
test_labels = [list(label) for label in test_labels]


In [ ]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)


In [ ]:
print(f"Length of train encodings: {len(train_encodings['input_ids'])}")
print(f"Length of train labels: {len(train_labels)}")


Length of train encodings: 2210
Length of train labels: 2210


In [ ]:
train_dataset = EmotionDataset(train_encodings, train_labels)
test_dataset = EmotionDataset(test_encodings, test_labels)


In [ ]:
# After filtering and splitting the data
print(f"Length of train encodings: {len(train_encodings['input_ids'])}")
print(f"Length of train labels: {len(train_labels)}")


Length of train encodings: 2210
Length of train labels: 2210


In [ ]:
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)  # Ensure this is based on labels


In [ ]:
train_dataset = EmotionDataset(train_encodings, train_labels)
test_dataset = EmotionDataset(test_encodings, test_labels)


In [ ]:
# Ensure both lists are of the same length
min_length = min(len(train_encodings['input_ids']), len(train_labels))
train_encodings = {key: val[:min_length] for key, val in train_encodings.items()}
train_labels = train_labels[:min_length]


In [ ]:
trainer.train()


<ipython-input-14-137e8c813add>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss


IndexError: index 2212 is out of bounds for dimension 0 with size 2210

In [ ]:
# Prepare the data
texts = df['text'].tolist()
labels = df[['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']].values.tolist()

# Split into training and testing datasets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Filter out non-string entries and ensure all entries are strings
train_texts = [str(text) for text in train_texts if isinstance(text, (str, bytes))]
test_texts = [str(text) for text in test_texts if isinstance(text, (str, bytes))]

# Check the content and lengths
print("Filtered Train Texts Length:", len(train_texts))
print("Filtered Test Texts Length:", len(test_texts))

# Tokenize the train and test texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')


Filtered Train Texts Length: 2210
Filtered Test Texts Length: 553


In [ ]:
# Tokenize the train and test texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')


In [ ]:
import torch

# Convert labels to tensors
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)


In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset instances
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                       # the instantiated 🤗 Transformers model to be trained
    args=training_args,                # training arguments, defined above
    train_dataset=train_dataset,       # training dataset
    eval_dataset=test_dataset          # evaluation dataset
)


In [ ]:
trainer.train()


ValueError: Expected input batch_size (16) to match target batch_size (80).

In [ ]:
print("Train encodings:", train_encodings['input_ids'].shape)
print("Train labels shape:", train_labels.shape)
print("Length of train encodings:", len(train_encodings['input_ids']))
print("Length of train labels:", len(train_labels))


Train encodings: torch.Size([2210, 100])
Train labels shape: torch.Size([2214, 5])
Length of train encodings: 2210
Length of train labels: 2214


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]  # This should correctly match the size
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
# Example of accessing the model's output shape
outputs = model(input_ids=..., attention_mask=...)
print("Model output shape:", outputs.logits.shape)


AttributeError: 'ellipsis' object has no attribute 'size'

In [ ]:
# Assuming `train_dataset` is your dataset and it's properly set up
sample = train_dataset[0]  # Get the first item from the dataset

# Access the input_ids and attention_mask
input_ids = sample['input_ids'].unsqueeze(0)  # Add a batch dimension
attention_mask = sample['attention_mask'].unsqueeze(0)  # Add a batch dimension

# Pass to the model
outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Print the model output shape
print("Model output shape:", outputs.logits.shape)


Model output shape: torch.Size([1, 5])


In [ ]:
# Example input for prediction
sample_input = train_dataset[0]  # Use a sample from your dataset
input_ids = sample_input['input_ids'].unsqueeze(0)  # Add batch dimension
attention_mask = sample_input['attention_mask'].unsqueeze(0)  # Add batch dimension

# Get model outputs
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Get predicted class (argmax of logits)
predicted_class = torch.argmax(outputs.logits, dim=1)
print("Predicted class:", predicted_class.item())


Predicted class: 4


In [ ]:
print(type(y_true), type(y_pred))
print(len(y_true), len(y_pred))


<class 'list'> <class 'list'>
1 1


In [ ]:
import numpy as np

y_true = np.array(y_true).flatten()  # Flatten in case it's multi-dimensional
y_pred = np.array(y_pred).flatten()


In [ ]:
print(y_pred.shape)  # Check the shape of y_pred



(1,)


In [ ]:
if y_pred.ndim == 1:
    # y_pred is already in the correct format
    pass  # You can use it directly
elif y_pred.ndim == 2:
    # y_pred has two dimensions
    y_pred = np.argmax(y_pred, axis=1)  # Use axis=1 if y_pred has shape (n_samples, n_classes)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming you have true labels and predictions
# Replace these with your actual data
y_true = [...]  # Your true labels
y_pred = [...]  # Your predicted classes or probabilities

# Ensure they are NumPy arrays
y_true = np.array(y_true).flatten()
y_pred = np.array(y_pred)

# Check the shape of y_pred
print("Shape of y_pred:", y_pred.shape)

# If y_pred is one-hot encoded or probabilities, convert to class indices
if y_pred.ndim == 1:
    # y_pred is already in the correct format
    pass  # You can use it directly
elif y_pred.ndim == 2:
    # y_pred has two dimensions
    y_pred = np.argmax(y_pred, axis=1)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(5), yticklabels=range(5))
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


Shape of y_pred: (1,)


ValueError: unknown is not supported

In [ ]:
print("Shape of y_true:", np.array(y_true).shape)
print("Shape of y_pred:", np.array(y_pred).shape)


Shape of y_true: (1,)
Shape of y_pred: (1,)


In [ ]:
test_texts = [
    "This is the first test sentence.",
    "Here is another example sentence.",
    "This sentence is for testing the model."
]


In [ ]:
print(type(test_texts))  # Should be <class 'list'>
print(test_texts)  # Check the contents


<class 'list'>
['This is the first test sentence.', 'Here is another example sentence.', 'This sentence is for testing the model.']


In [ ]:
# Assuming you have your test texts stored in a list
test_texts = ["This is the first test sentence.", "Here is another test sentence."]

# Ensure it's a list of strings
if isinstance(test_texts, (str, list)):
    # Your processing code here
    print("Test texts are valid.")
else:
    print("Test texts should be a string or a list of strings.")


Test texts are valid.


In [ ]:
# Tokenize the test texts
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')


In [ ]:
# Example test texts (ensure this is correctly formatted)
test_texts = [
    "This is the first test sentence.",
    "Here is another example sentence.",
    "This sentence is for testing the model."
]

# Check the format
if isinstance(test_texts, str):
    test_texts = [test_texts]  # Convert to list if a single string
elif not isinstance(test_texts, list):
    raise ValueError("test_texts must be a list of strings.")

# Tokenize the test texts
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')


In [ ]:
# Ensure that your model and tokenizer are already defined
# model = ... (Load your model)
# tokenizer = ... (Load your tokenizer)

# Step 1: Pass the tokenized test data through the model
with torch.no_grad():  # Disable gradient calculations for inference
    outputs = model(**test_encodings)  # Use the unpacked dictionary for inputs

# Step 2: Process predictions
logits = outputs.logits  # Access the output logits
y_pred = torch.argmax(logits, dim=-1)  # Get the predicted class indices

# Convert predictions to numpy array for further analysis if needed
y_pred = y_pred.numpy()

# If you have true labels (y_true), you can calculate confusion matrix or accuracy
# y_true = [...]  # Your true labels for the test data
# cm = confusion_matrix(y_true, y_pred)  # Calculate confusion matrix

# Print predictions
print("Predicted classes:", y_pred)


Predicted classes: [0 0 0]


In [ ]:
# Check logits for debugging
logits = outputs.logits
print("Logits:", logits.numpy())

# Also print probabilities
probabilities = torch.softmax(logits, dim=-1)
print("Probabilities:", probabilities.numpy())


Logits: [[ 0.25306934  0.01621219 -0.11319974  0.12930952 -0.09374641]
 [ 0.262191    0.03036144 -0.12702811  0.13670495 -0.10961349]
 [ 0.2630216   0.03777031 -0.12458492  0.10345572 -0.09073339]]
Probabilities: [[0.24552244 0.19374274 0.1702247  0.21694167 0.17356853]
 [0.24738854 0.19619922 0.16762696 0.21821354 0.17057168]
 [0.24804321 0.19801676 0.1683418  0.21146026 0.174138  ]]
